In [27]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
import numpy as np

In [28]:
def statevector_to_dirac(statevec, tol=1e-8):
    """
    Return a readable Dirac-style string for a Statevector (complex amplitudes).
    e.g. 0.7071|00> + 0.7071|11>
    """
    n = int(np.log2(len(statevec)))
    terms = []
    for i, amp in enumerate(statevec):
        if abs(amp) < tol:
            continue
        # basis label: binary with n bits, big-endian (q0 is leftmost)
        label = "|" + format(i, '0{}b'.format(n)) + ">"
        # format amplitude: show real/imag as needed
        a = amp
        if abs(a.imag) < tol:
            # purely real
            s_amp = f"{a.real:.6f}"
        elif abs(a.real) < tol:
            s_amp = f"{a.imag:.6f}j"
        else:
            s_amp = f"({a.real:.6f}{a.imag:+.6f}j)"
        terms.append(f"{s_amp}{label}")
    if not terms:
        return "0"
    return " + ".join(terms)

In [29]:
def simulate_and_print(qc, label):
    sv = Statevector.from_instruction(qc)
    print(f"\n--- {label} ---")
    print("Circuit (text):")
    print(qc.draw('text'))
    print("Statevector (raw complex array):")
    print(np.round(sv.data, 6))
    print("Statevector (Dirac form):")
    print(statevector_to_dirac(sv.data))
    return sv

***Part 1: Define HSHT (single qubit)***

In [30]:
hsht = QuantumCircuit(1, name='HSHT')
hsht.h(0)
hsht.s(0)
hsht.h(0)
hsht.tdg(0)

print("HSHT circuit (text):")
print(hsht.draw('text'))

HSHT circuit (text):
   ┌───┐┌───┐┌───┐┌─────┐
q: ┤ H ├┤ S ├┤ H ├┤ Tdg ├
   └───┘└───┘└───┘└─────┘


*Apply HSHT to |0>*

In [31]:
qc0 = QuantumCircuit(1)
qc0.append(hsht.to_instruction(), [0])
sv0 = simulate_and_print(qc0, "HSHT applied to |0>")


--- HSHT applied to |0> ---
Circuit (text):
   ┌──────┐
q: ┤ HSHT ├
   └──────┘
Statevector (raw complex array):
[0.5+0.5j      0. -0.707107j]
Statevector (Dirac form):
(0.500000+0.500000j)|0> + -0.707107j|1>


*Apply HSHT to |1>*

In [ ]:
qc1 = QuantumCircuit(1)
qc1.x(0)            # prepare |1>
qc1.append(hsht.to_instruction(), [0])
sv1 = simulate_and_print(qc1, "HSHT applied to |1>")

***Part 2: Reversibility check***

In [32]:
rev_qc = QuantumCircuit(1)
rev_qc.append(hsht.to_instruction(), [0])
rev_qc.append(hsht.to_instruction().inverse(), [0])  # inverse of the HSHT block
sv_rev = simulate_and_print(rev_qc, "HSHT followed by HSHT^-1 (reversibility check)")

rev_qc_with_init = QuantumCircuit(1)
rev_qc_with_init.append(hsht.to_instruction(), [0])
rev_qc_with_init.append(hsht.to_instruction().inverse(), [0])
sv_rev2 = simulate_and_print(rev_qc_with_init, "HSHT -> HSHT^-1 starting from |0> (should be |0>)")


--- HSHT followed by HSHT^-1 (reversibility check) ---
Circuit (text):
   ┌──────┐┌─────────┐
q: ┤ HSHT ├┤ HSHT_dg ├
   └──────┘└─────────┘
Statevector (raw complex array):
[1.+0.j 0.+0.j]
Statevector (Dirac form):
1.000000|0>

--- HSHT -> HSHT^-1 starting from |0> (should be |0>) ---
Circuit (text):
   ┌──────┐┌─────────┐
q: ┤ HSHT ├┤ HSHT_dg ├
   └──────┘└─────────┘
Statevector (raw complex array):
[1.+0.j 0.+0.j]
Statevector (Dirac form):
1.000000|0>


***Part 3: Bell state circuit (H on qubit 0, CNOT(0->1))***

In [34]:
def bell_from_input(a, b):
    """Create circuit preparing |a b>, then H on q0 and CNOT(0,1)."""
    qc = QuantumCircuit(2)
    if a == 1:
        qc.x(0)
    if b == 1:
        qc.x(1)
    qc.h(0)
    qc.cx(0, 1)
    return qc

print("\n--- Bell circuit base ---")
base_bell = QuantumCircuit(2)
base_bell.h(0)
base_bell.cx(0,1)
print(base_bell.draw('text'))
simulate_and_print(base_bell, "Bell circuit applied to |00> (default) -> should produce |Φ+>")


--- Bell circuit base ---
     ┌───┐     
q_0: ┤ H ├──■──
     └───┘┌─┴─┐
q_1: ─────┤ X ├
          └───┘

--- Bell circuit applied to |00> (default) -> should produce |Φ+> ---
Circuit (text):
     ┌───┐     
q_0: ┤ H ├──■──
     └───┘┌─┴─┐
q_1: ─────┤ X ├
          └───┘
Statevector (raw complex array):
[0.707107+0.j 0.      +0.j 0.      +0.j 0.707107+0.j]
Statevector (Dirac form):
0.707107|00> + 0.707107|11>
Statevector([0.70710678+0.j, 0.        +0.j, 0.        +0.j,
             0.70710678+0.j],
            dims=(2, 2))


In [35]:
inputs = [(0,0), (0,1), (1,0), (1,1)]
for a,b in inputs:
    qc = bell_from_input(a,b)
    label = f"Bell circuit with input |{a}{b}>"
    sv = simulate_and_print(qc, label)

    bell_states = {
        "Phi_plus":  np.array([1,0,0,1]) / np.sqrt(2),
        "Phi_minus": np.array([1,0,0,-1]) / np.sqrt(2),
        "Psi_plus":  np.array([0,1,1,0]) / np.sqrt(2),
        "Psi_minus": np.array([0,1,-1,0]) / np.sqrt(2),
    }


--- Bell circuit with input |00> ---
Circuit (text):
     ┌───┐     
q_0: ┤ H ├──■──
     └───┘┌─┴─┐
q_1: ─────┤ X ├
          └───┘
Statevector (raw complex array):
[0.707107+0.j 0.      +0.j 0.      +0.j 0.707107+0.j]
Statevector (Dirac form):
0.707107|00> + 0.707107|11>

--- Bell circuit with input |01> ---
Circuit (text):
     ┌───┐     
q_0: ┤ H ├──■──
     ├───┤┌─┴─┐
q_1: ┤ X ├┤ X ├
     └───┘└───┘
Statevector (raw complex array):
[0.      +0.j 0.707107+0.j 0.707107+0.j 0.      +0.j]
Statevector (Dirac form):
0.707107|01> + 0.707107|10>

--- Bell circuit with input |10> ---
Circuit (text):
     ┌───┐┌───┐     
q_0: ┤ X ├┤ H ├──■──
     └───┘└───┘┌─┴─┐
q_1: ──────────┤ X ├
               └───┘
Statevector (raw complex array):
[ 0.707107+0.j  0.      +0.j  0.      +0.j -0.707107+0.j]
Statevector (Dirac form):
0.707107|00> + -0.707107|11>

--- Bell circuit with input |11> ---
Circuit (text):
     ┌───┐┌───┐     
q_0: ┤ X ├┤ H ├──■──
     ├───┤└───┘┌─┴─┐
q_1: ┤ X ├─────┤ X ├
     └─

***Compare state up to global phase:***

In [36]:
    best = None
    best_overlap = 0
    v = sv.data
    for name, bs in bell_states.items():
        overlap = abs(np.vdot(bs, v))
        if overlap > best_overlap:
            best_overlap = overlap
            best = name
    print(f"  -> Best matching Bell state: {best} (overlap = {best_overlap:.6f})")

  -> Best matching Bell state: Psi_minus (overlap = 1.000000)


***SUMMARY:***
 - HSHT is a composition of unitary gates (H, S, H, Tdg) and therefore unitary and reversible.
 - Reversibility check returned the initial state (HSHT then HSHT^-1 -> identity).
 - H + CNOT acting on computational basis inputs { |00>,|01>,|10>,|11> } produces the four Bell states (|Φ+>,|Ψ+>,|Φ->,|Ψ->) up to known local/global phases.